## RAG Agent

In [1]:
from dotenv import load_dotenv
import os
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage
from operator import add as add_messages
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.tools import tool
from IPython.display import Image, display

load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o-mini", temperature = 0) # To minimize hallucination - temperature = 0 makes the model output more deterministic 

### Embedding Model - has to also be compatible with the LLM

In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",)

pdf_path = "Stock_Market_Performance_2024.pdf"

# Safety measure for debugging purposes 
if not os.path.exists(pdf_path):
    raise FileNotFoundError(f"PDF file not found: {pdf_path}")

pdf_loader = PyPDFLoader(pdf_path) 

# Checks if the PDF is there
try:
    pages = pdf_loader.load()
    print(f"PDF has been loaded and has {len(pages)} pages")
except Exception as e:
    print(f"Error loading PDF: {e}")
    raise

# Chunking 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

pages_split = text_splitter.split_documents(pages) # We now apply this to our pages

PDF has been loaded and has 9 pages


### Create tools

In [6]:
persist_directory = r"C:\Users\MILTON\Documents\Training\data"
collection_name = "stock_market"

# If our collection does not exist in the directory, we create using the os command
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)


try:
    # Here, we actually create the chroma database using our embeddigns model
    vectorstore = Chroma.from_documents(
        documents=pages_split,
        embedding=embeddings,
        persist_directory=persist_directory,
        collection_name=collection_name
    )
    print(f"Created ChromaDB vector store!")
    
except Exception as e:
    print(f"Error setting up ChromaDB: {str(e)}")
    raise


# Now we create our retriever 
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5} # K is the amount of chunks to return
)

@tool
def retriever_tool(query: str) -> str:
    """
    This tool searches and returns the information from the Stock Market Performance 2024 document.
    """

    docs = retriever.invoke(query)

    if not docs:
        return "I found no relevant information in the Stock Market Performance 2024 document."
    
    results = []
    for i, doc in enumerate(docs):
        results.append(f"Document {i+1}:\n{doc.page_content}")
    
    return "\n\n".join(results)


tools = [retriever_tool]

llm = llm.bind_tools(tools)

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]


def Should_continue(state: AgentState):
    """Check if the last message contains tool calls."""
    result = state['messages'][-1]
    return hasattr(result, 'tool_calls') and len(result.tool_calls) > 0


system_prompt = """
You are an intelligent AI assistant who answers questions about Stock Market Performance in 2024 based on the PDF document loaded into your knowledge base.
Use the retriever tool available to answer questions about the stock market performance data. You can make multiple calls if needed.
If you need to look up some information before asking a follow up question, you are allowed to do that!
Please always cite the specific parts of the documents you use in your answers.
"""

# Creating a dictionary of our tools
tools_dict = {our_tool.name: our_tool for our_tool in tools} 

Created ChromaDB vector store!


### Create agents

In [10]:
# LLM Agent
def Call_llm(state: AgentState) -> AgentState:
    """Function to call the LLM with the current state."""
    messages = list(state['messages'])
    messages = [SystemMessage(content=system_prompt)] + messages
    message = llm.invoke(messages)
    return {'messages': [message]}


# Retriever Agent
def Retriever(state: AgentState) -> AgentState:
    """Execute tool calls from the LLM's response."""

    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
        print(f"Calling Tool: {t['name']} with query: {t['args'].get('query', 'No query provided')}")
        
        if not t['name'] in tools_dict: # Checks if a valid tool is present
            print(f"\nTool: {t['name']} does not exist.")
            result = "Incorrect Tool Name, Please Retry and Select tool from List of Available tools."
        
        else:
            result = tools_dict[t['name']].invoke(t['args'].get('query', ''))
            print(f"Result length: {len(str(result))}")
            

        # Appends the Tool Message
        results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))

    print("Tools Execution Complete. Back to the model!")
    return {'messages': results}

### Build graph

In [12]:
graph = StateGraph(AgentState)
graph.add_node("call_llm", Call_llm)
graph.add_node("retriever", Retriever)

graph.add_conditional_edges(
    "call_llm",
    Should_continue,
    {True: "retriever", False: END}
)
graph.add_edge("retriever", "call_llm")
graph.set_entry_point("call_llm")

rag_agent = graph.compile()

In [13]:
print("\n=== RAG AGENT===")
    
while True:
    user_input = input("\nWhat is your question: ")
    if user_input.lower() in ['exit', 'quit']:
        break
            
    messages = [HumanMessage(content=user_input)] # converts back to a HumanMessage type

    result = rag_agent.invoke({"messages": messages})
        
    print("\n=== ANSWER ===")
    print(result['messages'][-1].content)


=== RAG AGENT===



What is your question:  how many messages?


Calling Tool: retriever_tool with query: number of messages
Result length: 4185
Tools Execution Complete. Back to the model!

=== ANSWER ===
The document does not provide a specific count of messages. It primarily discusses stock market performance in 2024, focusing on various companies and their financial metrics. If you have a specific question about the stock market performance or any related topic, feel free to ask!



What is your question:  exit
